Objectifs :

- Appliquer la méthode de sous-échantillonnage "downsampling" pour traiter les données déséquilibrées de la variable cible "grav",
- Réduire la dimensionnalité des données en utilisant l'analyse en Composantes Principales (PCA) avec deux composantes,
- Utiliser Recursive Feature Elimination (RFE) pour sélectionner les 15 caractéristiques les plus importantes,
- Entraîner un modèle d'arbre de décision en utilisant la technique de Random Forest,
- Mesurer le temps d'entraînement du modèle et calculer les métriques.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

from sklearn.preprocessing import OneHotEncoder, LabelEncoder

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, f1_score
from sklearn.ensemble import RandomForestClassifier

In [ ]:
df = pd.read_csv('fusion3.csv', low_memory=False)

In [ ]:
df['date'] = pd.to_datetime(df['date'])
df['month'] = df['date'].dt.month
df['day'] = df['date'].dt.day
df = df.drop(['Unnamed: 0','num_acc','an_nais','an_naiss','age_acc_an','num_veh','senc','occutc','permis','secuDeux','date'], axis=1)
df['place'] = df['place'].astype('object')
df = df.dropna()

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2291739 entries, 0 to 2291796
Data columns (total 35 columns):
 #   Column              Dtype  
---  ------              -----  
 0   place               object 
 1   catu                object 
 2   grav                object 
 3   sexe                object 
 4   trajet              object 
 5   locp                object 
 6   actp                object 
 7   etatp               object 
 8   secuUn              object 
 9   tranches_ages       object 
 10  catr                object 
 11  circ                object 
 12  nbv                 float64
 13  vosp                object 
 14  prof                object 
 15  plan                object 
 16  surf                object 
 17  infra               object 
 18  situ                object 
 19  obs                 object 
 20  obsm                object 
 21  choc                object 
 22  manv                object 
 23  catv_Label          object 
 24  lum                 obje

<h1>Encodage des variables</h1>

Les variables sont encodées une à une afin de pallier à un déficit de mémoire sur certaines machines.

In [ ]:
df = pd.get_dummies(df, columns=['catu'])

In [ ]:
le = LabelEncoder()
df['sexe'] = le.fit_transform(df['sexe'])

In [ ]:
df = pd.get_dummies(df, columns = ['trajet'])

In [ ]:
# One-Hot Encoding
df = pd.get_dummies(df, columns=['locp'])

In [ ]:
# One-Hot Encoding
df = pd.get_dummies(df, columns=['actp'])

In [ ]:
df = pd.get_dummies(df, columns = ['etatp'])

In [ ]:
# One-Hot Encoding
df = pd.get_dummies(df, columns=['secuUn'])

In [ ]:
df = pd.get_dummies(df, columns = ['tranches_ages'])

In [ ]:
# One-Hot Encoding
df = pd.get_dummies(df, columns=['catr'])

In [ ]:
# One-Hot Encoding
df = pd.get_dummies(df, columns=['circ'])

In [ ]:
# Binary Encoding
df['vosp'] = df['vosp'].apply(lambda x: 0 if x == 'Sans objet(0)' else 1)

In [ ]:
# One-Hot Encoding
df = pd.get_dummies(df, columns=['prof'])

In [ ]:
# One-Hot Encoding
df = pd.get_dummies(df, columns=['plan'])

In [ ]:
# One-Hot Encoding
df = pd.get_dummies(df, columns=['surf'])

In [ ]:
# One-Hot Encoding
df = pd.get_dummies(df, columns=['infra'])

In [ ]:
# One-Hot Encoding
df = pd.get_dummies(df, columns=['situ'])

In [ ]:
# One-Hot Encoding
df = pd.get_dummies(df, columns=['obs'])

In [ ]:
# One-Hot Encoding
df = pd.get_dummies(df, columns=['obsm'])

In [ ]:
# Regrouper la catégorie "Non renseigné" avec "Aucun"
df['choc'] = df['choc'].replace('Non renseigné', 'Aucun')

# Effectuer le one-hot encoding
df = pd.get_dummies(df, columns=['choc'])

In [ ]:
frequency_encoding = df['manv'].value_counts(normalize=True)
df['manv'] = df['manv'].map(frequency_encoding)

In [ ]:
frequency_encoding = df['catv_Label'].value_counts(normalize=True)
df['catv_Label'] = df['catv_Label'].map(frequency_encoding)

In [ ]:
frequency_encoding = df['dep'].value_counts(normalize=True)
df['dep'] = df['dep'].map(frequency_encoding)

In [ ]:
frequency_encoding = df['com'].value_counts(normalize=True)
df['com'] = df['com'].map(frequency_encoding)

In [ ]:
df = pd.get_dummies(df, columns = ['lum'])

In [ ]:
df['agg'] = df['agg'].replace({'En agglomération': 1, 'Hors agglomération': 0})

In [ ]:
df = pd.get_dummies(df, columns = ['int'])

In [ ]:
df = pd.get_dummies(df, columns = ['atm'])

In [ ]:
df = pd.get_dummies(df, columns = ['col'])

In [ ]:
df = pd.get_dummies(df, columns = ['jour_de_la_semaine'])

In [ ]:
df.shape

(2291739, 195)

In [ ]:
df.columns

Index(['place', 'grav', 'sexe', 'nbv', 'vosp', 'manv', 'catv_Label', 'agg',
       'com', 'dep',
       ...
       'col_Sans collision',
       'col_Trois véhicules et plus - collisions multiples',
       'col_Trois véhicules et plus – en chaîne',
       'jour_de_la_semaine_Dimanche', 'jour_de_la_semaine_Jeudi',
       'jour_de_la_semaine_Lundi', 'jour_de_la_semaine_Mardi',
       'jour_de_la_semaine_Mercredi', 'jour_de_la_semaine_Samedi',
       'jour_de_la_semaine_Vendredi'],
      dtype='object', length=195)

<h1>Rééquilibrage de classe</h1>

In [ ]:
# Affichage des différentes valeurs de la colonne "grav"
unique_grav_values = df['grav'].unique()
print("Valeurs uniques de la colonne grav :", unique_grav_values)

Valeurs uniques de la colonne grav : ['Blessé léger' 'Blessé hospitalisé' 'Indemne' 'Tué']


In [ ]:
# Vérification des  proportions de chaque niveau de gravité
df['grav'].value_counts()

Indemne               937743
Blessé léger          825286
Blessé hospitalisé    467533
Tué                    61177
Name: grav, dtype: int64

In [ ]:
from sklearn.utils import resample

# Comptage des occurrences de chaque classe de la variable cible "grav"
class_counts = df['grav'].value_counts()

# Identification de la classe majoritaire
majority_class = class_counts.idxmax()

# Calcul du nombre d'occurrences souhaité pour les autres classes de la variable cible "grav"
target_count = class_counts.min()  # Utilisation de la taille de la classe minoritaire

# Sous-échantillonnage pour chaque classe
undersampled_data = []
for cls in class_counts.index:
    cls_data = df[df['grav'] == cls]
    undersampled_cls_data = resample(cls_data, replace=False, n_samples=target_count, random_state=42)
    undersampled_data.append(undersampled_cls_data)

# Concaténation des données sous-échantillonnées
undersampled_data = pd.concat(undersampled_data)

# Mélange des données pour garantir l'ordre aléatoire
undersampled_data = undersampled_data.sample(frac=1, random_state=42)

In [ ]:
# Vérification des nouvelles proportions de chaque niveau de gravité
undersampled_data['grav'].value_counts()

Blessé hospitalisé    61177
Blessé léger          61177
Tué                   61177
Indemne               61177
Name: grav, dtype: int64

In [ ]:
df = undersampled_data

In [ ]:
# Vérification s'il y a des NaN dans le DataFrame df
nan_values = df.isna().any()

print("Colonnes avec des valeurs NaN :")
print(nan_values[nan_values].index)

Colonnes avec des valeurs NaN :
Index([], dtype='object')


In [ ]:
df.head(10)

,place,grav,sexe,nbv,vosp,manv,catv_Label,agg,com,dep,...,col_Sans collision,col_Trois véhicules et plus - collisions multiples,col_Trois véhicules et plus – en chaîne,jour_de_la_semaine_Dimanche,jour_de_la_semaine_Jeudi,jour_de_la_semaine_Lundi,jour_de_la_semaine_Mardi,jour_de_la_semaine_Mercredi,jour_de_la_semaine_Samedi,jour_de_la_semaine_Vendredi
1632800,1.0,Blessé hospitalisé,0,4.0,0,0.455666,0.651106,0,0.007251,0.061163,...,0,0,0,0,0,1,0,0,0,0
1152753,1.0,Blessé léger,0,2.0,0,0.082017,0.651106,0,0.000805,0.010663,...,0,0,0,0,0,0,1,0,0,0
1635636,1.0,Blessé léger,0,2.0,0,0.020620,0.651106,1,0.034947,0.061163,...,0,0,0,0,0,0,0,1,0,0
1609842,1.0,Blessé hospitalisé,1,1.0,0,0.455666,0.034623,0,0.000907,0.012446,...,0,0,0,0,0,0,0,0,0,1
1088968,3.0,Blessé hospitalisé,1,4.0,0,0.455666,0.651106,0,0.003245,0.007339,...,0,0,0,0,0,0,0,1,0,0
1872013,1.0,Tué,1,2.0,0,0.455666,0.651106,1,0.001367,0.009388,...,0,0,0,0,1,0,0,0,0,0
2038427,1.0,Blessé léger,1,2.0,0,0.455666,0.651106,0,0.002461,0.002910,...,0,0,0,0,0,1,0,0,0,0
381573,1.0,Tué,1,3.0,0,0.455666,0.071721,0,0.000931,0.006288,...,0,0,0,0,0,0,0,0,0,1
1003575,1.0,Indemne,1,2.0,0,0.455666,0.651106,1,0.001774,0.003138,...,0,0,0,0,1,0,0,0,0,0
552762,1.0,Blessé léger,1,4.0,0,0.082017,0.084295,1,0.034947,0.061163,...,0,0,0,0,0,1,0,0,0,0


<h1>Réduction de dimension PCA et Sélection des caractéristiques les plus importantces avec RFE</h1>

In [ ]:
X = df.drop('grav', axis=1)
y = df['grav']

In [ ]:
# Réduction de dimension avec PCA
from sklearn.decomposition import PCA
pca = PCA(n_components=2)
X_pca = pca.fit_transform(X)

In [ ]:
# Sélection des caractéristiques importantes avec RFE
from sklearn.feature_selection import RFE

model = RandomForestClassifier(random_state=42)
num_features_to_select = 15
rfe = RFE(estimator=model, n_features_to_select=num_features_to_select)
X_rfe = rfe.fit_transform(X_pca, y)
selected_columns = X_pca[:, rfe.support_]

In [ ]:
# Vérification des NaN dans X_rfe
nan_indices_x_rfe = np.isnan(X_rfe)
if np.any(nan_indices_x_rfe):
    print("Il y a des NaN dans X_rfe.")
    print("Indices des lignes avec NaN dans X_rfe:", np.where(nan_indices_x_rfe)[0])
else:
    print("Aucun NaN dans X_rfe.")

# Vérification des NaN dans y
nan_indices_y = y.isnull()
if np.any(nan_indices_y):
    print("Il y a des NaN dans y.")
    print("Indices des lignes avec NaN dans y:", np.where(nan_indices_y)[0])
else:
    print("Aucun NaN dans y.")


Aucun NaN dans X_rfe.
Aucun NaN dans y.


<h1>Entrainement du modèle</h1>

In [ ]:
# Division des données en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X_rfe, y, test_size=0.2, random_state=42)

In [ ]:
# Entrainement du modèle
from time import time

t0 = time()
model.fit(X_train, y_train)
t1 = time() - t0

print("Réalisé en {} secondes".format(round(t1,3)))

Réalisé en 101.523 secondes


<h1>Calcul des métriques d'évaluation</h1>

In [ ]:
model.score(X_test, y_test)

0.29056842793510684

In [ ]:
model.score(X_train, y_train)

0.999621997691121

In [ ]:
t0 = time()
y_pred = model.predict(X_test)
t1 = time() - t0
print("Réalisé en {} secondes".format(round(t1,3)))

print(pd.crosstab(y_test, y_pred, rownames=['Classe réelle'], colnames=['Classe prédite']))
print(classification_report(y_test, y_pred))

Réalisé en 2.44 secondes
Classe prédite      Blessé hospitalisé  Blessé léger  Indemne   Tué
Classe réelle                                                      
Blessé hospitalisé                3169          3005     2980  3065
Blessé léger                      3059          3232     3165  2753
Indemne                           2929          3044     3896  2533
Tué                               3005          2631     2552  3924
                    precision    recall  f1-score   support

Blessé hospitalisé       0.26      0.26      0.26     12219
      Blessé léger       0.27      0.26      0.27     12209
           Indemne       0.31      0.31      0.31     12402
               Tué       0.32      0.32      0.32     12112

          accuracy                           0.29     48942
         macro avg       0.29      0.29      0.29     48942
      weighted avg       0.29      0.29      0.29     48942



In [ ]:
from imblearn.metrics import geometric_mean_score

geometric_mean_score(y_test, y_pred)

0.2891209402747039

In [ ]:
from sklearn.metrics import balanced_accuracy_score

balanced_accuracy_score(y_test, y_pred)

0.2905480099870789

In [ ]:
from sklearn.model_selection import StratifiedKFold, cross_val_score

cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

score = cross_val_score(model, X_pca, y, cv=cv, scoring='roc_auc_ovo', verbose=2, n_jobs = -1)

print(f"scores ROC AUC OvO pour chaque pli: {score}")
print(f"Moyenne des scores ROC AUC OvO : {score.mean()}")

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 out of   5 | elapsed:  3.2min remaining:  4.8min
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:  3.2min remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:  3.2min finished


scores ROC AUC OvO pour chaque pli: [0.5470895  0.54801875 0.54850045 0.54687734 0.54759278]
Moyenne des scores ROC AUC OvO : 0.547615765821446


In [ ]:
# Mémoire insuffisante donc utilisation d'un sous-ensemble de données pour réduire la taille
subset_size = 1000
X_pca_subset = X_pca[:subset_size]
y_subset = y[:subset_size]

# Calcul des scores avec le sous-ensemble de données
score = cross_val_score(model, X_pca_subset, y_subset, cv=cv, scoring='roc_auc_ovo', verbose=2, n_jobs=-1)

# Affichage des scores
print(f"scores ROC AUC OvO pour chaque pli: {score}")
print(f"Moyenne des scores ROC AUC OvO : {score.mean()}")


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 out of   5 | elapsed:    0.4s remaining:    0.6s


scores ROC AUC OvO pour chaque pli: [0.48409378 0.50912692 0.47019073 0.51209895 0.49427476]
Moyenne des scores ROC AUC OvO : 0.49395702818461784


[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    2.3s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    2.3s finished
